In [ ]:
import pandas as pd
import time
pd.options.mode.chained_assignment = None

# functions
def get_electorate():
    ''' select electorate from text menu '''       
    valid_response = False
    while not valid_response:
        try:
            print("\nWhich electorate do you wish to analyse?")
            for i, electorate in enumerate(electorates.electorate):
                print(f"{i + 1}. {electorate}")
            print("6. quit program")
            text = input("Enter a number from 1 to 6:")            
            text = int(text)
            if (text > 0) and (text < 7):
                valid_response = True
                return text
            else:
                print("That's not a valid choice.")
        except:
            print("That's not a valid choice.")

def eliminate_candidate():
    ''' remove eliminated candidate and return ID '''
    candidate = candidates.index[len(candidates) - 1]
    name = candidates.loc[candidate].cname
    party = candidates.loc[candidate].party
    primary = candidates.loc[candidate].primary    
    eliminated.loc[candidate] = [name, party, primary]
    print(f"{name} is eliminated!")
    return candidate

def elect_candidate(candidate):
    ''' declare elected candidate and calculate fractional vote '''
    global elected
    name = candidates.loc[candidate].cname
    print(f"{name} is elected!")
    party = candidates.loc[candidate].party
    primary = candidates.loc[candidate].primary    
    elected.loc[candidate] = [name, party, primary]
    # calculate fractional vote    
    fraction = (candidates.loc[candidate].votes - quota) / candidates.loc[candidate].votes
    print(f"{name} had {candidates.loc[candidate].votes:,.1f} votes. Fractional transfer value is {round(fraction, 6)}")
    return fraction

def distribute_votes(candidate, elected):
    ''' redistributes votes/surplus votes from eliminated/elected candidates '''
    global candidates, sample, exhausted
    print(f"redistributing {candidates.loc[candidate].cname}'s {'surplus ' if elected else ''}votes ...", end="\r")
    start = time.time()
    interval = 1
    elapsed = interval
    for i, indice in enumerate(sample.index):        
        # is vote for this candidate?
        if sample.loc[indice].votes[sample.loc[indice].pref] == candidate:            
            if elected:
                sample.at[indice, "value"] = round(sample.loc[indice].value * fraction, 6)
            # remove vote from eliminate/elected candidate
            candidates.at[candidate, "votes"] = candidates.loc[candidate, "votes"] - sample.loc[indice].value
            # is there a next preference?
            preference_resolved = False
            while not preference_resolved:
                if len(sample.loc[indice].votes) > sample.loc[indice].pref + 1:
                    pref = sample.loc[indice].pref + 1
                    sample.at[indice, "pref"] = pref
                    votes = sample.loc[indice].votes
                    value = sample.loc[indice].value                
                    # is preferenced candidate still in count?
                    if votes[pref] in candidates.index:
                        # add vote to preferenced candidate
                        candidates.at[votes[pref], "votes"] = candidates.loc[votes[pref]].votes + value
                # add to exhausted votes
                else:                
                    exhausted = exhausted + sample.loc[indice].value
                    sample.drop(index=indice, inplace=True)
                    preference_resolved = True
        if (time.time() - elapsed) > start:
            print(f"redistributing {candidates.loc[candidate].cname}'s {'surplus ' if elected else ''}votes ... {(i + 1) / len(sample):.1%}", end="\r")
            elapsed = elapsed + interval
    print(f"redistributing {candidates.loc[candidate].cname}'s {'surplus ' if elected else ''}votes ... complete")

# BEGIN PROGRAM

print("2020 ACT ELECTION SIMULATOR")
print("\u00a9 Markus Mannheim (ABC Canberra)")

# read in electorates
electorates = pd.read_csv("./data/Electorates.txt", index_col="ecode")

# begin cycle
while True:
    electorate = get_electorate()
    
    # user wants to quit
    if electorate == 6:
        break
    
    # load electorate data
    print(f"\nloading {electorates.loc[electorate].electorate} data ...", end="\r")
    candidates = pd.read_csv(f"./data/candidates_{electorates.loc[electorate].electorate}.csv", index_col="id")
    parties = pd.read_csv(f"./data/parties_{electorates.loc[electorate].electorate}.csv", index_col="id")
    votes = pd.read_csv(f"./data/votes_{electorates.loc[electorate].electorate}.csv", index_col="id")
    votes.votes = votes.votes.apply(lambda x: x.replace("[", "").replace("]", "").replace("'", "").split(", "))
    print(f"loading {electorates.loc[electorate].electorate} data ... complete")
    
    # create sample data to speed up calculation
    print("sampling data for analysis ...", end="\r")
    sample_size = 1
    sample = votes.sample(frac=sample_size)
    print("sampling data for analysis ... complete")
 
    # prepare datasets to contain elected and eliminated candidates
    print("preparing results containers ...", end="\r")
    elected = pd.DataFrame(columns=["cname", "party", "primary"])
    eliminated = pd.DataFrame(columns=["cname", "party", "primary"])
    exhausted = 0
    print("preparing results containers ... complete")

    # establish quota
    total_votes = len(sample)
    quota = int(total_votes / 6) + 1
    print(f"\nquota for {electorates.loc[electorate].electorate}: {format(quota, ',')} votes")

    # record primary votes
    print("recording primary votes ...", end="\r")
    start = time.time()
    interval = 1
    elapsed = interval
    for i, indice in enumerate(sample.index):
        candidate = sample.loc[indice].votes[0]
        candidates.at[candidate, "votes"] = candidates.loc[candidate, "votes"] + 1
        if (time.time() - elapsed) > start:
            print(f"recording primary votes ... {(i + 1) / len(sample):.1%}", end="\r")
    candidates.primary = candidates.votes.apply(lambda x: f"{x / total_votes:.1%}")
    candidates.sort_values("votes", ascending=False, inplace=True)
    print("recording primary votes ... complete")
    
    # begin counting cycle
    count_no = 1
    while len(elected) < 5:
        print(f"\nCOUNT No. {count_no}\n")
        # check if candidates elected
        reached_quota = candidates[candidates["votes"] >= quota]
        if len(reached_quota) == 0:
            print("no candidates elected")
            eliminated_candidate = eliminate_candidate()
            distribute_votes(eliminated_candidate, False)
            candidates.drop(index=eliminated_candidate, inplace=True)
        else:
            i = 0
            while len(elected) < 5:
                candidate = reached_quota.index[i]
                fraction = elect_candidate(candidate)
                distribute_votes(candidate, True)
                candidates.drop(index=candidate, inplace=True)
                i = i + 1
                if len(reached_quota) < i + 1:
                    break
        candidates.sort_values("votes", ascending=False, inplace=True)
        count_no = count_no + 1        

    print()
    print(pd.concat([elected, candidates, eliminated]))

# exit program
print("\nEnjoy your day.")